In [21]:
# install required dependencies
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install yake

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [22]:
# import all the required dependencies
import kagglehub
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divyendu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# Download dataset
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")
print("Path to dataset files:", path)

dataset = pd.read_csv(path + "/mtsamples.csv")
print("Head: ", dataset.head)

Path to dataset files: /Users/divyendu/.cache/kagglehub/datasets/tboyle10/medicaltranscriptions/versions/1
Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

   

In [13]:
# data llmlmlmlmlmlm

dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

def clean_text(text):
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

dataset['transcription'] = dataset['transcription'].apply(clean_text)
dataset['medical_specialty'] = dataset['medical_specialty'].apply(clean_text)

In [14]:
specialty_counts = dataset['medical_specialty'].value_counts()
valid_specialties = specialty_counts[specialty_counts >= 50].index
dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]
categories = dataset.groupby(dataset["medical_specialty"])

for i, cat in enumerate(categories):
    category_name, category = cat
    print(f"Category {i}: {category_name}: {len(category)}")

Category 0: cardiovascular pulmonary: 371
Category 1: consult history phy: 516
Category 2: discharge summary: 108
Category 3: emergency room reports: 75
Category 4: ent otolaryngology: 96
Category 5: gastroenterology: 224
Category 6: general medicine: 259
Category 7: hematology oncology: 90
Category 8: nephrology: 81
Category 9: neurology: 223
Category 10: neurosurgery: 94
Category 11: obstetrics gynecology: 155
Category 12: office notes: 50
Category 13: ophthalmology: 83
Category 14: orthopedic: 355
Category 15: pain management: 61
Category 16: pediatrics neonatal: 70
Category 17: psychiatry psychology: 53
Category 18: radiology: 273
Category 19: soap chart progress notes: 166
Category 20: surgery: 1088
Category 21: urology: 156


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    dataset['transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

pipeline = Pipeline([
    ('tfidaataset', TfidfVectorizer(max_features=1000)),  # Convert text to TF-Idaataset features
    ('clf', MultinomialNB())  # Train a Naive Bayes classifier
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.3774
                           precision    recall  f1-score   support

 cardiovascular pulmonary       0.52      0.41      0.45        74
      consult history phy       0.29      0.89      0.44       103
        discharge summary       0.30      0.14      0.19        22
   emergency room reports       0.00      0.00      0.00        15
       ent otolaryngology       0.00      0.00      0.00        19
         gastroenterology       0.43      0.29      0.35        45
         general medicine       0.17      0.02      0.03        52
      hematology oncology       0.00      0.00      0.00        18
               nephrology       0.00      0.00      0.00        16
                neurology       0.41      0.36      0.38        45
             neurosurgery       0.00      0.00      0.00        19
    obstetrics gynecology       0.44      0.13      0.20        31
             office notes       0.00      0.00      0.00        10
            ophthalmology       0.00      0.

/Users/divyendu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/divyendu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/divyendu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [24]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization, Dropout
import yake

label_encoder = LabelEncoder()
dataset['encoded_label'] = label_encoder.fit_transform(dataset['medical_specialty'])
dataset.dropna(subset=['keywords'], inplace=True)

def extract_keywords(text):
    kw_extractor = yake.KeywordExtractor(n=1, top=50)  # Extract top 10 single-word keywords
    keywords = kw_extractor.extract_keywords(text)
    return ' '.join([kw[0] for kw in keywords])  # Return keywords as a space-separated string
# Apply keyword extraction
dataset['new_keywords'] = dataset['transcription'].apply(extract_keywords)

dataset['new_keywords'] = dataset['new_keywords'].apply(lambda x: ' '.join(x))
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    dataset['new_keywords'], dataset['encoded_label'], test_size=0.1, random_state=42, stratify=dataset['encoded_label']
)

max_words = 20000  # Increased vocabulary size
max_len = 300  # Increased sequence length
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)
# Build improved CNN model

model = Sequential([
    Embedding(input_dim=max_words, output_dim=256, input_length=max_len),
    Conv1D(256, 5, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    GlobalMaxPooling1D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train model
model.fit(X_train_seq, y_train, epochs=5, batch_size=64, validation_data=(X_test_seq, y_test))
# Evaluate model
loss, accuracy = model.evaluate(X_test_seq, y_test)
print(f'Accuracy: {accuracy:.4f}')

Epoch 1/5


/Users/divyendu/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 157ms/step - accuracy: 0.1824 - loss: 3.4791 - val_accuracy: 0.3534 - val_loss: 3.0453
Epoch 2/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 158ms/step - accuracy: 0.3106 - loss: 2.4766 - val_accuracy: 0.3726 - val_loss: 3.0287
Epoch 3/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 157ms/step - accuracy: 0.3258 - loss: 2.2976 - val_accuracy: 0.3452 - val_loss: 3.0011
Epoch 4/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 159ms/step - accuracy: 0.3815 - loss: 2.1720 - val_accuracy: 0.3726 - val_loss: 2.9638
Epoch 5/5
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 161ms/step - accuracy: 0.3625 - loss: 2.0901 - val_accuracy: 0.3644 - val_loss: 2.8991
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3643 - loss: 2.9044
Accuracy: 0.3644


In [25]:
# Combine transcription and sample name for keyword extraction
dataset['combined_text'] = dataset['cleaned_transcription'] + " " + dataset['cleaned_sample_name']

# YAKE Keyword Extraction
def extract_keywords(text):
    kw_extractor = yake.KeywordExtractor(n=2, top=10)  # Extract top 10 bi-gram keywords
    keywords = kw_extractor.extract_keywords(text)
    return ' '.join([kw[0] for kw in keywords])  # Return keywords as a space-separated string

# Apply YAKE for keyword extraction
dataset['keywords'] = dataset['combined_text'].apply(extract_keywords)
# Encode labels
label_encoder = LabelEncoder()
dataset['encoded_label'] = label_encoder.fit_transform(dataset['medical_specialty'])
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    dataset['keywords'], dataset['encoded_label'], test_size=0.2, random_state=42, stratify=dataset['encoded_label']
)

# Tokenization
max_words = 20000  # Increased vocabulary size
max_len = 50  # Reduced sequence length since keywords are shorter
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

# Build improved CNN model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=256, input_length=max_len),
    Conv1D(128, 5, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train model
model.fit(X_train_seq, y_train, epochs=10, batch_size=128, validation_data=(X_test_seq, y_test))
# Evaluate model
loss, accuracy = model.evaluate(X_test_seq, y_test)
print(f'Accuracy: {accuracy:.4f}')

KeyError: 'cleaned_transcription'

In [26]:

# Tokenization
max_words = 20000  # Increased vocabulary size
max_len = 500  # Reduced sequence length since keywords are shorter
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

# Build improved CNN model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=256, input_length=max_len),
    Conv1D(128, 5, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train model
model.fit(X_train_seq, y_train, epochs=20, batch_size=128, validation_data=(X_test_seq, y_test))
# Evaluate model
loss, accuracy = model.evaluate(X_test_seq, y_test)
print(f'Accuracy: {accuracy:.4f}')

NameError: name 'l2' is not defined